In [10]:
# !pip install deeppavlov
# !pip install transformers

In [2]:
from deeppavlov import configs, build_model

ner_model = build_model(configs.ner.ner_rus_bert_torch, download=True)

/usr/local/lib/python3.7/dist-packages/cryptography/hazmat/backends/openssl/x509.py:17: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  utils.DeprecatedIn35,
2022-02-12 10:20:41.315 INFO in 'deeppavlov.core.data.utils'['utils'] at line 95: Downloading from http://files.deeppavlov.ai/0.16/ner/ner_rus_bert_torch.tar.gz to /root/.deeppavlov/ner_rus_bert_torch.tar.gz
100%|██████████| 1.43G/1.43G [00:46<00:00, 30.7MB/s]
2022-02-12 10:21:28.605 INFO in 'deeppavlov.core.data.utils'['utils'] at line 272: Extracting /root/.deeppavlov/ner_rus_bert_torch.tar.gz archive into /root/.deeppavlov/models
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package perluniprops to /root/nltk_data...
[nltk_data]   Unzipping misc/perlunip

Downloading:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/642 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.57M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

2022-02-12 10:22:00.487 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /root/.deeppavlov/models/ner_rus_bert_torch/tag.dict]


Downloading:   0%|          | 0.00/681M [00:00<?, ?B/s]

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initializ

In [8]:
article_text = 'РОСТОВ-НА-ДОНУ, 2 фев – РИА Новости. Тело мужчины, сбежавшего более двух месяцев назад из Сарпинского психоневрологического диспансера в Калмыкии, обнаружили в поселке неподалеку, сообщает СУСК России по Калмыкии. По информации следствия, вечером 29 ноября мужчина самовольно покинул психдиспансер. По факту безвестного исчезновения мужчины было возбуждено уголовное дело по статье "Убийство". Был организован его розыск."В дополнение к ранее опубликованной информации о розыске подопечного БУРК "Сарпинский психоневрологический диспансер" Строкова А.Н. региональное следственное управление СК России сообщает, что на территории поселка Годжур Сарпинского района обнаружено тело мужчины", - говорится в сообщении.В настоящее время по данному факту организована проверка.'

In [5]:
from graph import InferenceWorker

In [6]:
class SampleNER(InferenceWorker):
    def __call__(self, t, d):
        if t == "text":
            yield "entities", self.ner_from_text(d), "NER"
        else:
            yield None        
    def ner_from_text(self, text):
        entities = []
        new_entity = None
        new_tag = None
        for entity, tag in zip(result[0][0], result[1][0]):
            if tag == 'O':
                if new_entity != None:
                    entities.append([new_entity, new_tag])
                    new_tag = None
                    new_entity = None     
            else:
                if tag[0] == 'B': 
                    if new_entity != None:
                        entities.append([new_entity, new_tag])
                    new_entity = entity
                    new_tag = tag[2:]
                if tag[0] == 'I':
                    new_entity += (' ' + entity)
        if new_entity != None:
            entities.append([new_entity, new_tag])
        return entities
        

In [7]:
worker = SampleNER()
worker.serialize()

Worker saved!


In [9]:
for x in worker("text", article_text):
    print(x)

('entities', [['РОСТОВ - НА - ДОНУ', 'LOC'], ['РИА Новости', 'ORG'], ['Сарпинского психоневрологического диспансера', 'ORG'], ['Калмыкии', 'LOC'], ['СУСК', 'ORG'], ['России', 'LOC'], ['Калмыкии', 'LOC'], ['БУРК " Сарпинский психоневрологический диспансер "', 'ORG'], ['Строкова А . Н', 'PER'], ['СК', 'ORG'], ['России', 'LOC'], ['Годжур', 'LOC'], ['Сарпинского района', 'LOC']], 'NER')
